# Ejercicio 9

Utilizando los ejemplos del archivo AUTOS.csv genere un modelo utilizando un multiperceptrón para predecir el precio del auto (atributo price) y la cantidad de millas por galón en ruta (MPG-highway) en función del resto de los atributos.  

Recuerde completar los valores faltantes, utilizar normalización y dividir el dataset en entrenamiento y validación (80/20).  

Realice 20 ejecuciones independientes de cada configuración seleccionada calculando las épocas promedio y el error cuadrático medio (ECM).  

Analice los resultados y respalde las afirmaciones referidas a los resultados obtenidos.  
Utilice un máximo de 1000 épocas con lotes de 50 e implemente una parada temprana con paciencia de 15.


In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

DATA_PATH = "./Data/"

# ...existing code...
# leer marcando '?' como NA
data = pd.read_csv(DATA_PATH + 'autos.csv', na_values=['?'])

data = data.replace('?', np.nan)
# columnas numéricas a forzar y completar
cols = ["normalized-losses", "bore", "stroke", "horsepower", "peak-rpm", "price"]

for col in cols:
    if col in data.columns:
        data[col] = pd.to_numeric(data[col], errors="coerce")
        data[col] = data[col].fillna(data[col].mean())

# CODIFICAR columnas categóricas como números comenzando en 1
cat_cols = data.select_dtypes(include=['object', 'category']).columns.tolist()
for col in cat_cols:
    # rellenar NaN por una etiqueta concreta y factorizar
    data[col] = data[col].fillna('missing')
    data[col] = pd.factorize(data[col])[0] + 1  # códigos 0.. -> sumamos 1 para 1..

# ahora seleccionar por nombres de columna objetivo (más robusto que índices)
target_columns = ['highway-mpg', 'price']
if not set(target_columns).issubset(data.columns):
    raise ValueError("No se encontraron las columnas objetivo: " + ", ".join(target_columns))

T = data[target_columns].values
X = data.drop(columns=target_columns).values

data_scaler, target_scaler = StandardScaler(), StandardScaler()

X = data_scaler.fit_transform(X)
T = target_scaler.fit_transform(T)
# ...existing code...

# split 80/20
X_train, X_test, T_train, T_test = train_test_split(X, T, test_size=0.2, random_state=42, shuffle=True)



In [ ]:
import tensorflow as tf

# Verifica si hay GPUs disponibles y configura para usar CUDA si es posible
gpus = tf.config.list_physical_devices('GPU')
if gpus:
  try:
    # Configura TensorFlow para que use solo la memoria necesaria en la GPU
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    print("TensorFlow está configurado para usar CUDA (GPU).")
  except RuntimeError as e:
    print(e)
else:
  print("No se detectaron GPUs. TensorFlow usará la CPU.")



TensorFlow está configurado para usar CUDA (GPU).
Dispositivos físicos detectados:
PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')
PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')
GPU disponible: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


# Inciso A

Complete la siguiente tabla y realice un análisis de los valores obtenidos:

**Tabla 1 — Épocas promedio**

| Optimizador | tanh | sigmoid | ReLU | LeakyReLU |
| ----------- | ---- | ------- | ---- | --------- |
| SGD         |      |         |      |           |
| RMSProp     |      |         |      |           |
| Adam        |      |         |      |           |

**Tabla 2 — ECM promedio**

| Optimizador | tanh | sigmoid | ReLU | LeakyReLU |
| ----------- | ---- | ------- | ---- | --------- |
| SGD         |      |         |      |           |
| RMSProp     |      |         |      |           |
| Adam        |      |         |      |           |



In [9]:
from keras import Sequential
from keras.layers import Input, Dense

optimizers=['sgd','RMSProp','Adam']
functions=['tanh','sigmoid','relu','leaky_relu']

ENTRADAS = X_train.shape[1]

for opt in optimizers:
  for func in functions:
    epochs_list = []
    ecm_list = []
    for i in range(20):
      model=Sequential([
        Input(shape=[ENTRADAS]),
        Dense(8, activation=func),
        Dense(8, activation=func),
        Dense(2, activation='softmax')
      ])

      model.compile(optimizer=opt,loss='categorical_crossentropy',metrics=['accuracy', 'mse', 'mae'])
      
      early_stop = tf.keras.callbacks.EarlyStopping(
                  monitor='val_loss', patience=15, restore_best_weights=True
              )

      history = model.fit(X_train, T_train, epochs=500, batch_size=50,
                          validation_data=(X_test, T_test),
                          callbacks=[early_stop], verbose=0)

      # épocas usadas en esta ejecución
      epochs_run = len(history.history.get('loss', []))
      epochs_list.append(epochs_run)

      # ECM en el test set (MSE promedio sobre las 2 salidas)
      y_pred = model.predict(X_test, verbose=0)
      ecm = np.mean((y_pred - T_test)**2)
      ecm_list.append(ecm)

    # imprimir resumen para esta combinación (optimizador, función)
    print(f"opt={opt}  func={func} -> épocas promedio: {np.mean(epochs_list):.1f} ± {np.std(epochs_list):.1f}, ECM promedio: {np.mean(ecm_list):.4f} ± {np.std(ecm_list):.4f}")
# ...existing code...

opt=sgd  func=tanh -> épocas promedio: 330.3 ± 135.1, ECM promedio: 1.0242 ± 0.0456


KeyboardInterrupt: 